<a href="https://colab.research.google.com/github/cydal/Weather-App/blob/main/derin's_taikai_hackathon_edit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [70]:
import requests
import pandas as pd
import urllib.parse
from functools import reduce
import numpy as np

In [6]:
dataset = "era5_volumetric_soil_water_layer_1-hourly"
dataset = "era5_land_precip-hourly"
dataset = "cpcc_temp_max-daily"
#dataset = "era5_wind_100m_u-hourly"
lat, lon = 5.68999972, -0.20985164

In [7]:
query = f"https://api.dclimate.net/apiv2/grid-history/{dataset}/{lat}_{lon}?also_return_metadata=false&use_imperial_units=true&also_return_snapped_coordinates=true&convert_to_local_time=true"
response = requests.get(query)

In [31]:
def get_data(dataset, lat, lon):
  query = f"https://api.dclimate.net/apiv2/grid-history/{dataset}/{lat}_{lon}?also_return_metadata=false&use_imperial_units=true&also_return_snapped_coordinates=true&convert_to_local_time=true"
  response = requests.get(query)

  print(response.status_code)
  if response.status_code != 200:
    return("Error: Check location & dataset")

  data = response.json()
  first = list(data['data'].keys())[0]
  last = list(data['data'].keys())[-1]

  key, value = [], []
  for k in data['data']:
    key.append(k)
    # remove text 
    if data['data'][k]:
      value.append(data['data'][k].split(" ")[0])
    else:
      value.append(data['data'][k])

  df = pd.DataFrame({
                    "date_time": key,
                    f"{dataset}": value
  })
  df["date_time"] = pd.to_datetime(df["date_time"])


  return(df)

In [32]:
temp = get_data(dataset, lat, lon)

200


In [33]:
pd.to_datetime("2021-07-28 18:00:00").hour

18

In [34]:
address = 'Accra, Ghana'
url = 'https://nominatim.openstreetmap.org/search/' + urllib.parse.quote(address) +'?format=json'

response = requests.get(url).json()
print(response[0]["lat"])
print(response[0]["lon"])

5.5571096
-0.2012376


In [35]:
lat, lon = response[0]["lat"], response[0]["lon"]

In [36]:
def calc_wind(u, v):
  ws = np.sqrt(u**2 + v*82)
  wd = np.mod(180+ np.rad2deg(np.arctan2(u, v)), 360)
  return([ws, wd])

In [54]:
all_data[4]

'Error: Check location & dataset'

In [ ]:
df['wind_speed'] = df.apply (lambda x: calc_wind(row), axis=1)

In [55]:
datasets = ["era5_wind_100m_u-hourly", "era5_wind_100m_v-hourly", "era5_land_precip-hourly", 
            "era5_volumetric_soil_water_layer_1-hourly","era5_land_2m_temp-hourly", 
            "cpcc_temp_min-daily", "cpcc_temp_max-daily"]

In [56]:
all_data = []
for dataset in datasets:
  all_data.append(get_data(dataset, lat, lon))


def get_all_data(datasets, lat, lon):
  all_data = []

  for dataset in datasets:
    all_data.append(get_data(dataset, lat, lon))
  return(all_data)

200
200
200
200
200
200
200


In [57]:
df_merged_hourly = reduce(lambda  left,right: pd.merge(left,right,on=['date_time'],
                                            how='outer'), all_data[0:5]).fillna(0)
df_merged_temp_daily = reduce(lambda  left,right: pd.merge(left,right,on=['date_time'],
                                            how='outer'), all_data[-2:]).fillna(0)

In [58]:
df_merged_hourly.tail(3)

,date_time,era5_wind_100m_u-hourly,era5_wind_100m_v-hourly,era5_land_precip-hourly,era5_volumetric_soil_water_layer_1-hourly,era5_land_2m_temp-hourly
373099,2021-07-24 19:00:00+00:00,7.202934860415176,13.600572655690767,0,0.0296,0
373100,2021-07-24 20:00:00+00:00,7.426628489620616,12.057086614173228,0,0.0296,0
373101,2021-07-24 21:00:00+00:00,7.806907659269865,10.312276306370796,0,0.0296,0


In [59]:
df_merged_temp_daily.tail(3)

,date_time,cpcc_temp_min-daily,cpcc_temp_max-daily
15546,2021-07-25,73.922,86.18
15547,2021-07-26,73.004,84.70400000000001
15548,2021-07-27,74.426,80.348


In [61]:
query = "2021-07-24 19:00:00+00:00"
query2 = query.split(" ")[0]

In [42]:
if (df_merged_hourly.date_time == pd.Timestamp(query)).any():
    print ('date exist')  
else:
  print("Not in date")

date exist


In [79]:
hour_temp = df_merged_hourly[df_merged_hourly["date_time"]==query]["era5_land_2m_temp-hourly"].iloc[0]
min_temp = df_merged_temp_daily[df_merged_temp_daily["date_time"]==query2]["cpcc_temp_min-daily"].iloc[0]
max_temp = df_merged_temp_daily[df_merged_temp_daily["date_time"]==query2]["cpcc_temp_max-daily"].iloc[0]
soil_water_layer = df_merged_hourly[df_merged_hourly["date_time"]==query]["era5_volumetric_soil_water_layer_1-hourly"].iloc[0]
wind_u = float (df_merged_hourly[df_merged_hourly["date_time"]==query]["era5_wind_100m_u-hourly"].iloc[0])
wind_v = float (df_merged_hourly[df_merged_hourly["date_time"]==query]["era5_wind_100m_v-hourly"].iloc[0])
wind_vel = calc_wind(wind_u,wind_v)[0]
wind_dir = calc_wind(wind_u,wind_v)[1]
precipitation = df_merged_hourly[df_merged_hourly["date_time"]==query]["era5_land_precip-hourly"].iloc[0]
print(hour_temp, "\n", min_temp, "\n", max_temp, "\n",soil_water_layer, "\n", wind_vel, "\n",wind_dir, "\n", precipitation)

0 
 74.642 
 86.95400000000001 
 0.0296 
 34.163273092167664 
 207.9059299064315 
 0


temp

In [81]:
def rain_classifier(value):
  if value == 0:
    print("No rain in the last hour")
  elif value > 0 and value<=0.05:
    print("It's drizzling a, an umbrella won't be a bad idea")
  elif value > 0.5 and value<=0.1:
    print("A light rain, you should go out with an umbrella")
  elif value > 0.1 and value<=0.3:
    print("Rain's quite heavy")
  else:
    print("Very heavy downpour, hope you are indoors")

In [82]:
rain_classifier(precipitation)

No rain in the last hour


In [ ]:
new_temp = temp.copy()

Rain: Precipitation in the form of liquid water drops that have diameters greater than 0.5 millimeter; if less than 0.5 millimeter, it is drizzle. Rain and drizzle are the only forms of liquid precipitation. Rain is classified as light, meaning rain falling at a rate between a trace and 0.10 inch per hour; moderate, 0.11 to 0.30 inch per hour; heavy, more than 0.30 inch per hour. Drizzle is classified as light, falling at a rate from a trace to 0.01 inch per hour; moderate, 0.01 to 0.02 inch per hour; heavy, greater than 0.02 inch per hour.